# Lightning AI SDXL LoRA Trainer

Este cuaderno prepara un entorno dentro de Lightning AI (ruta raíz ``/teamspace/studios/this_studio``)
y ejecuta el entrenador de LoRA para Stable Diffusion XL incluido en este fork.

Los pasos principales son:

1. Crear/usar un entorno gestionado por [uv](https://github.com/astral-sh/uv).
2. Introducir un nombre de LoRA; el cuaderno creará `/teamspace/studios/this_studio/Loras/<nombre>/` con las subcarpetas
   `dataset/` y `output/`. Copia tus imágenes y `.txt` dentro de `dataset/`. Puedes definir *activation tags* globales y
   habilitar el mezclado de tags desde la configuración.
3. Instalar las dependencias necesarias.
4. Definir la configuración de entrenamiento (epochs, repeats, tasas de aprendizaje, tags, modelo base y VAE a descargar automáticamente, etc.).
5. Ejecutar el entrenamiento calculando automáticamente los *steps* mediante la fórmula requerida.
6. Seleccionar optimizadores personalizados (AdamW8bit, Prodigy, DAdapt*, Lion, etc.) y schedulers (`cosine`, `rex`,
   `cosine_with_restarts`, entre otros) incluidos en este fork.
7. Activa o desactiva los valores recomendados de cada optimizador con `use_optimizer_recommended_args` si quieres aplicar los presets sugeridos (Adafactor, AdamW8bit, Prodigy y CAME) sin tocar manualmente `optimizer_kwargs`.
8. Define el nombre final del archivo LoRA con `lora_name` o la bandera `--lora-name` para controlar el `.safetensors` generado.
9. Decide si quieres descargar el modelo base en formato Diffusers o como `.safetensors` con `load_diffusers_format` y, para los presets que lo necesiten (como NoobAI), fuerza o desactiva `v_prediction` desde la configuración o la CLI.



In [ ]:
from pathlib import Path
import os
import subprocess
import sys

LIGHTNING_ROOT = Path("/teamspace/studios/this_studio")
UV_BIN = Path.home() / ".local" / "bin" / "uv"

if not UV_BIN.exists():
    print("Instalando uv…")
    subprocess.run(
        ["/bin/bash", "-lc", "curl -LsSf https://astral.sh/uv/install.sh | sh"],
        check=True,
    )
else:
    print("uv ya está instalado")

os.environ["PATH"] = f"{UV_BIN.parent}:{os.environ['PATH']}"
os.environ.setdefault("UV_PROJECT_ENVIRONMENT", str(LIGHTNING_ROOT / ".venv"))
print("Entorno de uv:", os.environ["UV_PROJECT_ENVIRONMENT"])

repo_root = Path.cwd().resolve()
if not (repo_root / "lightning_ai").exists():
    candidate = repo_root / "LoRA_Easy_Training_scripts_Backend"
    if candidate.exists():
        repo_root = candidate.resolve()
if not (repo_root / "lightning_ai").exists():
    raise FileNotFoundError("No se encontró la carpeta 'lightning_ai' junto al notebook.")

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print("Repositorio detectado en:", repo_root)


In [ ]:
import subprocess

required_packages = [
    "numpy<2",
    "bitsandbytes==0.44.0",
    "prodigyopt",
    "lion-pytorch==0.0.6",
    "dadaptation",
    "pytorch-optimizer==3.1.2",
    "xformers==0.0.22.post7",
    "torch>=2.1",
    "torchvision",
    "accelerate>=0.23",
    "diffusers[torch]==0.25",
    "transformers==4.35",
    "peft>=0.7",
    "safetensors==0.4.4",
    "huggingface-hub==0.24.5",
    "toml==0.10.2",
    "tensorboard",
    "pytorch-lightning==1.9.0",
    "opencv-python==4.8.1.78",
]
print("Instalando dependencias con uv…")
subprocess.run(["uv", "pip", "install", "--upgrade", *required_packages], check=True)


In [ ]:
from pathlib import Path
from lightning_ai.sdxl_lora_trainer import (
    BASE_MODEL_PRESETS,
    VAE_PRESETS,
    TrainingConfig,
)

lora_name = "mi_lora_personalizado"

lora_workspace = (LIGHTNING_ROOT / "Loras" / lora_name).resolve()
dataset_workspace = lora_workspace / "dataset"
output_workspace = lora_workspace / "output"
dataset_workspace.mkdir(parents=True, exist_ok=True)
output_workspace.mkdir(parents=True, exist_ok=True)

base_config = TrainingConfig(
    dataset_dir=dataset_workspace,  # carpeta con imágenes y .txt emparejados
    output_dir=output_workspace,
    num_epochs=2,
    batch_size=1,
    gradient_accumulation=1,
    num_repeats=2,
    resolution=1024,
    network_rank=64,
    network_alpha=128,
    base_model="Stable Diffusion XL 1.0 base",
    load_diffusers_format=True,  # cambia a False para descargar checkpoints .safetensors
    v_prediction=None,  # usa True/False para forzar o desactivar v-prediction manualmente
    vae_model="Made by Ollin SDXL VAE FP16 Fix",
    unet_lr=1e-4,
    text_encoder_lr=5e-6,
    optimizer_type="adamw",
    weight_decay=1e-2,
    optimizer_beta1=0.9,
    optimizer_beta2=0.999,
    optimizer_eps=1e-8,
    optimizer_momentum=0.9,
    scheduler_type="cosine",
    lr_warmup_steps=None,
    scheduler_first_cycle_steps=None,
    scheduler_cycle_multiplier=1.0,
    scheduler_gamma=1.0,
    scheduler_min_lr=1e-6,
    scheduler_d=0.9,
    scheduler_power=1.0,
    train_text_encoders=True,
    mixed_precision="fp16",
    shuffle_tags=True,
    activation_tags=("mi_lora",),
    use_optimizer_recommended_args=True,
    cross_attention_backend="xformers",
    lora_name=lora_name,  # la extensión .safetensors se añade si falta
)

print("Directorio de trabajo del LoRA:", lora_workspace)
print("Modelos base disponibles (Diffusers / Safetensors):")
for name, preset in BASE_MODEL_PRESETS.items():
    print(f"  - {name}: diffusers={preset.diffusers_id} | safetensors={preset.single_file_url}")

print("VAEs disponibles (Diffusers / Safetensors):")
for name, preset in VAE_PRESETS.items():
    print(f"  - {name}: diffusers={preset.diffusers_id} | safetensors={preset.single_file_url}")

config = base_config.normalised_paths()
print("Configuración resuelta:")
for key, value in config.__dict__.items():
    print(f"  {key}: {value}")


In [ ]:
import json
from lightning_ai.sdxl_lora_trainer import FolderCaptionDataset, calculate_total_steps

preview_dataset = FolderCaptionDataset(
    dataset_dir=config.dataset_dir,
    resolution=config.resolution,
    activation_tags=config.activation_tags,
    shuffle_tags=config.shuffle_tags,
)

num_images = len(preview_dataset)
steps = calculate_total_steps(
    num_images=num_images,
    num_repeats=config.num_repeats,
    num_epochs=config.num_epochs,
    batch_size=config.batch_size,
)
print(json.dumps(
    {
        "num_images": num_images,
        "num_repeats": config.num_repeats,
        "num_epochs": config.num_epochs,
        "batch_size": config.batch_size,
        "calculated_steps": steps,
    },
    indent=2,
))


In [ ]:
from lightning_ai.sdxl_lora_trainer import train

print("Iniciando entrenamiento…")
train(config)